In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
#getting Counties Data
counties = pd.read_csv('us-counties.csv')
#counties['NAME'] = counties[['county','state' ]].agg(', '.join, axis=1)
startdate = '2020-01-01'
enddate = '2020-12-31'

mask = (counties['date'] >= startdate) & (counties['date'] <= enddate)

counties = counties.loc[mask].copy().reset_index(drop=True)
counties = counties.query("state != 'Puerto Rico' and state != 'Virgin Islands' and state != 'Northern Mariana Islands' and county != 'Unknown'")
counties = counties.groupby(['county','state']).max()
counties_clean = counties.drop(['date'], axis=1)

In [3]:
#getting age data
demo = pd.read_csv('countiesage.csv', usecols = ['STNAME', 'YEAR', 'CTYNAME', 'STATE', 'COUNTY', 'AGE65PLUS_TOT'], encoding_errors='ignore')
demo = demo.rename({('STNAME'): 'state', ('CTYNAME'): 'county'}, axis='columns')
#parsing the data
demo['county'] = demo['county'].str.split()
for i in range(len(demo['county'])):
  if demo['county'][i][-1] == 'County':
    del demo['county'][i][-1]
  elif demo['county'][i][-1] == 'Parish':
    del demo['county'][i][-1]
  elif demo['county'][i][-1] == 'Municipality':
    del demo['county'][i][-1]
demo['county'] = demo['county'].str.join(' ')
demo = demo.query("YEAR == 2").reset_index(drop=True)

In [4]:
#merging
merged = counties_clean.merge(demo, how='right', on=['state', 'county'])
merged = merged.query("fips == fips and cases == cases").reset_index(drop=True)
merged

,state,county,fips,cases,deaths,STATE,COUNTY,YEAR,AGE65PLUS_TOT
0,Alabama,Autauga,1001.0,4190.0,48.0,1,1,2,9273
1,Alabama,Baldwin,1003.0,13601.0,161.0,1,3,2,49125
2,Alabama,Barbour,1005.0,1514.0,32.0,1,5,2,4960
3,Alabama,Bibb,1007.0,1834.0,46.0,1,7,2,3691
4,Alabama,Blount,1009.0,4641.0,63.0,1,9,2,10927
...,...,...,...,...,...,...,...,...,...
3125,Wyoming,Sweetwater,56037.0,2966.0,16.0,56,37,2,5628
3126,Wyoming,Teton,56039.0,2138.0,4.0,56,39,2,3789
3127,Wyoming,Uinta,56041.0,1558.0,7.0,56,41,2,3117
3128,Wyoming,Washakie,56043.0,780.0,19.0,56,43,2,1728


In [5]:
#getting demographic data
demo2 = pd.read_csv('countiesdemo.csv', usecols = ['STNAME', 'YEAR', 'CTYNAME', 'STATE', 'COUNTY','TOT_POP', 'WA_MALE', 'WA_FEMALE','BA_MALE','BA_FEMALE', 'IA_MALE','IA_FEMALE', 'AA_MALE','AA_FEMALE', 'NA_MALE','NA_FEMALE'], encoding_errors='ignore')
demo2 = demo2.rename({('STNAME'): 'state', ('CTYNAME'): 'county'}, axis='columns')
#parsing the data
demo2['county'] = demo2['county'].str.split()
for i in range(len(demo2['county'])):
  if demo2['county'][i][-1] == 'County':
    del demo2['county'][i][-1]
  elif demo2['county'][i][-1] == 'Parish':
    del demo2['county'][i][-1]
  elif demo2['county'][i][-1] == 'Municipality':
    del demo2['county'][i][-1]
demo2['county'] = demo2['county'].str.join(' ')
demo2 = demo2.query("YEAR == 2").reset_index(drop=True)
demo2

,STATE,COUNTY,state,county,YEAR,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE
0,1,1,Alabama,Autauga,2,58877,21970,22610,5719,6417,134,147,302,388,35,31
1,1,1,Alabama,Autauga,2,3480,1297,1162,385,391,4,4,31,22,3,3
2,1,1,Alabama,Autauga,2,3683,1344,1287,414,391,13,9,22,29,0,4
3,1,1,Alabama,Autauga,2,4180,1528,1452,505,469,7,11,18,26,1,2
4,1,1,Alabama,Autauga,2,3832,1387,1390,460,421,6,3,21,24,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59712,56,45,Wyoming,Weston,2,557,279,225,3,0,3,0,19,20,0,0
59713,56,45,Wyoming,Weston,2,365,191,168,0,0,1,1,0,1,0,0
59714,56,45,Wyoming,Weston,2,222,105,110,0,0,0,2,0,0,0,0
59715,56,45,Wyoming,Weston,2,170,66,101,0,0,1,1,0,0,0,0


In [6]:
demo2 = demo2.drop(['STATE', 'COUNTY', 'YEAR'], axis=1)

In [7]:
new = demo2.groupby(['state', 'county']).sum()

In [8]:
#merging
merged2 = merged.merge(new, how='left', on=['state', 'county'])
merged2 = merged2.query("fips == fips and cases == cases").reset_index(drop=True)
merged2

,state,county,fips,cases,deaths,STATE,COUNTY,YEAR,AGE65PLUS_TOT,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE
0,Alabama,Autauga,1001.0,4190.0,48.0,1,1,2,9273,117754,43940,45220,11438,12834,268,294,604,776,70,62
1,Alabama,Baldwin,1003.0,13601.0,161.0,1,3,2,49125,466280,198954,208628,19640,20782,1928,1760,2076,3046,142,178
2,Alabama,Barbour,1005.0,1514.0,32.0,1,5,2,4960,50360,13082,11582,12894,11380,216,144,102,130,72,40
3,Alabama,Bibb,1007.0,1834.0,46.0,1,7,2,3691,44446,17438,16596,5848,3574,124,90,48,64,46,12
4,Alabama,Blount,1009.0,4641.0,63.0,1,9,2,10927,118162,56228,56832,1086,964,402,364,230,218,86,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,Wyoming,Sweetwater,56037.0,2966.0,16.0,56,37,2,5628,84316,41130,38000,652,488,654,584,366,476,64,64
3126,Wyoming,Teton,56039.0,2138.0,4.0,56,39,2,3789,46694,23110,21158,228,158,246,220,310,412,40,36
3127,Wyoming,Uinta,56041.0,1558.0,7.0,56,41,2,3117,40882,19922,19130,134,120,298,318,76,124,28,26
3128,Wyoming,Washakie,56043.0,780.0,19.0,56,43,2,1728,15316,7454,7056,48,32,122,136,52,76,6,4


In [9]:
#renaming variables
merged2['oldper'] = merged2['AGE65PLUS_TOT']/merged2['TOT_POP']
merged2['wper'] = (merged2['WA_MALE'] + merged2['WA_FEMALE'])/merged2['TOT_POP']
merged2['bper'] = (merged2['BA_MALE'] + merged2['BA_FEMALE'])/merged2['TOT_POP']
merged2['iper'] = (merged2['IA_MALE'] + merged2['IA_FEMALE'])/merged2['TOT_POP']
merged2['aper'] = (merged2['AA_MALE'] + merged2['AA_FEMALE'])/merged2['TOT_POP']
merged2['nper'] = (merged2['NA_MALE'] + merged2['NA_FEMALE'])/merged2['TOT_POP']
merged2 = merged2.drop(['WA_MALE', 'WA_FEMALE','BA_MALE','BA_FEMALE', 'IA_MALE','IA_FEMALE', 'AA_MALE','AA_FEMALE', 'NA_MALE','NA_FEMALE'], axis=1)

In [10]:
hpi_dta = pd.read_excel('housing.xlsx', usecols= ['Five-Digit ZIP Code', 'Year', 'HPI with 2000 base'], skiprows= 6)

In [11]:
#making hpi variables
hpilag = hpi_dta.query("Year == 2020").reset_index(drop= True)
hpinow = hpi_dta.query("Year == 2021").reset_index(drop= True)
hpilag = hpilag.rename({('Five-Digit ZIP Code'): 'fips', ('HPI with 2000 base'): 'hpilag'}, axis='columns')
hpinow = hpinow.rename({('Five-Digit ZIP Code'): 'fips', ('HPI with 2000 base'): 'hpinow'}, axis='columns')

In [12]:
#merging
merged3 = merged2.merge(hpilag, how='left', on='fips')
merged3['hpilag'] = pd.to_numeric(merged3['hpilag'], errors='coerce')
merged3 = merged3.merge(hpinow, how='left', on='fips')
merged3['hpinow'] = pd.to_numeric(merged3['hpinow'], errors='coerce')
merged3 = merged3.query("hpilag == hpilag and hpinow == hpinow").reset_index(drop= True)
merged3

,state,county,fips,cases,deaths,STATE,COUNTY,YEAR,AGE65PLUS_TOT,TOT_POP,oldper,wper,bper,iper,aper,nper,Year_x,hpilag,Year_y,hpinow
0,Alabama,Autauga,1001.0,4190.0,48.0,1,1,2,9273,117754,0.078749,0.757172,0.206125,0.004773,0.011719,0.001121,2020.0,181.80,2021.0,202.20
1,Alabama,Barbour,1005.0,1514.0,32.0,1,5,2,4960,50360,0.098491,0.489754,0.482010,0.007149,0.004607,0.002224,2020.0,167.88,2021.0,201.05
2,Alabama,Bibb,1007.0,1834.0,46.0,1,7,2,3691,44446,0.083045,0.765738,0.211988,0.004815,0.002520,0.001305,2020.0,182.98,2021.0,200.79
3,Alabama,Butler,1013.0,1508.0,45.0,1,13,2,3962,38090,0.104017,0.519559,0.450092,0.004201,0.014229,0.000420,2020.0,188.09,2021.0,212.92
4,Alabama,Clay,1027.0,1141.0,34.0,1,27,2,2969,28386,0.104594,0.833369,0.139083,0.005496,0.002677,0.000000,2020.0,202.36,2021.0,217.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,Wyoming,Carbon,56007.0,946.0,15.0,56,7,2,2564,28998,0.088420,0.934616,0.012484,0.021726,0.010277,0.001104,2020.0,135.05,2021.0,144.79
857,Wyoming,Crook,56011.0,388.0,7.0,56,11,2,1535,14356,0.106924,0.963082,0.008638,0.011702,0.002647,0.000139,2020.0,157.65,2021.0,175.24
858,Wyoming,Fremont,56013.0,3968.0,64.0,56,13,2,7540,78354,0.096230,0.733874,0.006203,0.223294,0.005284,0.000485,2020.0,139.22,2021.0,153.07
859,Wyoming,Hot Springs,56017.0,231.0,1.0,56,17,2,1266,9244,0.136954,0.946127,0.007356,0.019905,0.006058,0.000649,2020.0,161.19,2021.0,182.33


In [13]:
#getting region data
region = pd.read_csv('regions.csv', usecols=['State', 'Division'])
region = region.rename({('State'): 'state'}, axis='columns')
region

,state,Division
0,Alaska,Pacific
1,Alabama,East South Central
2,Arkansas,West South Central
3,Arizona,Mountain
4,California,Pacific
5,Colorado,Mountain
6,Connecticut,New England
7,District of Columbia,South Atlantic
8,Delaware,South Atlantic
9,Florida,South Atlantic


In [14]:
#merging final data
final = merged3.merge(region, how= 'left', on='state')
final['covidrate'] = final['deaths']/final['TOT_POP']
final

,state,county,fips,cases,deaths,STATE,COUNTY,YEAR,AGE65PLUS_TOT,TOT_POP,...,bper,iper,aper,nper,Year_x,hpilag,Year_y,hpinow,Division,covidrate
0,Alabama,Autauga,1001.0,4190.0,48.0,1,1,2,9273,117754,...,0.206125,0.004773,0.011719,0.001121,2020.0,181.80,2021.0,202.20,East South Central,0.000408
1,Alabama,Barbour,1005.0,1514.0,32.0,1,5,2,4960,50360,...,0.482010,0.007149,0.004607,0.002224,2020.0,167.88,2021.0,201.05,East South Central,0.000635
2,Alabama,Bibb,1007.0,1834.0,46.0,1,7,2,3691,44446,...,0.211988,0.004815,0.002520,0.001305,2020.0,182.98,2021.0,200.79,East South Central,0.001035
3,Alabama,Butler,1013.0,1508.0,45.0,1,13,2,3962,38090,...,0.450092,0.004201,0.014229,0.000420,2020.0,188.09,2021.0,212.92,East South Central,0.001181
4,Alabama,Clay,1027.0,1141.0,34.0,1,27,2,2969,28386,...,0.139083,0.005496,0.002677,0.000000,2020.0,202.36,2021.0,217.62,East South Central,0.001198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,Wyoming,Carbon,56007.0,946.0,15.0,56,7,2,2564,28998,...,0.012484,0.021726,0.010277,0.001104,2020.0,135.05,2021.0,144.79,Mountain,0.000517
857,Wyoming,Crook,56011.0,388.0,7.0,56,11,2,1535,14356,...,0.008638,0.011702,0.002647,0.000139,2020.0,157.65,2021.0,175.24,Mountain,0.000488
858,Wyoming,Fremont,56013.0,3968.0,64.0,56,13,2,7540,78354,...,0.006203,0.223294,0.005284,0.000485,2020.0,139.22,2021.0,153.07,Mountain,0.000817
859,Wyoming,Hot Springs,56017.0,231.0,1.0,56,17,2,1266,9244,...,0.007356,0.019905,0.006058,0.000649,2020.0,161.19,2021.0,182.33,Mountain,0.000108


In [15]:
#summary stats
final[['hpinow', 'covidrate', 'hpilag', 'Division', 'oldper', 'wper', 'bper', 'iper', 'aper', 'nper']].describe()

,hpinow,covidrate,hpilag,oldper,wper,bper,iper,aper,nper
count,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000,861.000000
mean,182.174262,0.000583,164.216585,0.097027,0.842637,0.082766,0.030591,0.018019,0.002383
std,44.770678,0.000451,41.411214,0.025035,0.167808,0.137474,0.094542,0.035895,0.018017
min,86.330000,0.000000,69.130000,0.023782,0.077583,0.000000,0.000000,0.000000,0.000000
25%,154.450000,0.000253,140.210000,0.080523,0.802642,0.008863,0.004799,0.005284,0.000373
50%,176.110000,0.000483,158.410000,0.096389,0.910807,0.021577,0.008872,0.008433,0.000772
75%,197.970000,0.000794,177.120000,0.111775,0.949567,0.087627,0.017322,0.015416,0.001593
max,566.080000,0.002801,536.470000,0.237805,0.986557,0.865831,0.862968,0.420419,0.487805


In [16]:
#regression without serial correlation
reg = smf.ols('hpinow ~ covidrate + C(Division) + oldper + wper + bper + iper + aper + nper',data=final).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 hpinow   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     15.09
Date:                Mon, 17 Apr 2023   Prob (F-statistic):           8.16e-35
Time:                        23:18:15   Log-Likelihood:                -4392.2
No. Observations:                 861   AIC:                             8816.
Df Residuals:                     845   BIC:                             8892.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           455.6739    103.686      4.395      0.000     252.161     659.187
C(Division)[T.East South Central]    10.8619      6.055      1.794      0.073      -1.022      22.746
C(Division)[T.Middle Atlantic]      -29.8987      9.349     -3.198      0.001     -48.250     -11.548
C(Division)[T.Mountain]               3.0437      5.562      0.547      0.584      -7.873      13.961
C(Division)[T.New England]           23.2997     11.937      1.952      0.051      -0.130      46.730
C(Division)[T.Pacific]              -12.8115      6.985     -1.834      0.067     -26.522       0.899
C(Division)[T.South Atlantic]        10.9717      6.284      1.746      0.081      -1.363      23.307
C(Division)[T.West North Central]    29.3681      5.158      5.694      0.000      19.245      39.491
C(Division)[T.West South Central]   -31.8484      5.271     -6.042      0.000     -42.195     -21.502
covidrate                         -3440.7749   3530.278     -0.975      0.330   -1.04e+04    3488.369
oldper                              -57.9868     63.193     -0.918      0.359    -182.020      66.046
wper                               -272.8680    104.810     -2.603      0.009    -478.587     -67.149
bper                               -252.0542    105.379     -2.392      0.017    -458.890     -45.218
iper                               -294.2033    110.560     -2.661      0.008    -511.207     -77.199
aper                               -340.7914    134.025     -2.543      0.011    -603.852     -77.731
nper                               -332.6317    167.686     -1.984      0.048    -661.761      -3.502
==============================================================================
Omnibus:                      500.138   Durbin-Watson:                   0.578
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6501.043
Skew:                           2.384   Prob(JB):                         0.00
Kurtosis:                      15.589   Cond. No.                     3.53e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.53e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
#regression with serial correlation
reg2 = smf.ols('hpinow ~ covidrate + C(Division) + oldper + wper + bper + iper + aper + nper + hpilag',data=final).fit()
reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 hpinow   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     2739.
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        00:15:28   Log-Likelihood:                -2785.8
No. Observations:                 861   AIC:                             5606.
Df Residuals:                     844   BIC:                             5686.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             6.3805     16.240      0.393      0.695     -25.495      38.256
C(Division)[T.East South Central]     1.4833      0.939      1.579      0.115      -0.360       3.326
C(Division)[T.Middle Atlantic]       -2.1279      1.456     -1.462      0.144      -4.985       0.729
C(Division)[T.Mountain]               4.8378      0.861      5.616      0.000       3.147       6.529
C(Division)[T.New England]            0.6812      1.853      0.368      0.713      -2.955       4.318
C(Division)[T.Pacific]               -0.0549      1.084     -0.051      0.960      -2.183       2.073
C(Division)[T.South Atlantic]         3.7311      0.974      3.830      0.000       1.819       5.643
C(Division)[T.West North Central]     1.6176      0.813      1.990      0.047       0.022       3.213
C(Division)[T.West South Central]    -0.3437      0.834     -0.412      0.680      -1.980       1.293
covidrate                           978.6053    547.266      1.788      0.074     -95.556    2052.767
oldper                              -15.5357      9.790     -1.587      0.113     -34.750       3.679
wper                                  0.9317     16.299      0.057      0.954     -31.060      32.924
bper                                  3.8209     16.379      0.233      0.816     -28.327      35.969
iper                                 -0.1757     17.196     -0.010      0.992     -33.928      33.576
aper                                 -3.9996     20.836     -0.192      0.848     -44.896      36.897
nper                                  1.2182     26.032      0.047      0.963     -49.878      52.314
hpilag                                1.0610      0.006    185.432      0.000       1.050       1.072
==============================================================================
Omnibus:                       82.826   Durbin-Watson:                   1.503
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              361.319
Skew:                           0.327   Prob(JB):                     3.47e-79
Kurtosis:                       6.106   Cond. No.                     4.38e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.38e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
#regression without categorical variables
reg3 = smf.ols('hpinow ~ covidrate + oldper + wper + bper + iper + aper + nper + hpilag',data=final).fit()
reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 hpinow   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     5084.
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        23:18:16   Log-Likelihood:                -2821.2
No. Observations:                 861   AIC:                             5660.
Df Residuals:                     852   BIC:                             5703.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6480     15.944     -0.103      0.918     -32.942      29.646
covidrate    505.6623    526.236      0.961      0.337    -527.209    1538.534
oldper        -5.3773      9.706     -0.554      0.580     -24.428      13.673
wper           8.3099     16.152      0.514      0.607     -23.392      40.012
bper          13.0886     16.191      0.808      0.419     -18.690      44.868
iper          10.5198     17.153      0.613      0.540     -23.148      44.187
aper          -0.3907     21.117     -0.019      0.985     -41.838      41.056
nper           7.2141     26.379      0.273      0.785     -44.562      58.990
hpilag         1.0695      0.005    200.585      0.000       1.059       1.080
==============================================================================
Omnibus:                       68.201   Durbin-Watson:                   1.405
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              193.506
Skew:                           0.387   Prob(JB):                     9.57e-43
Kurtosis:                       5.190   Cond. No.                     4.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
#F-test
(.981-.979)*(len(final)-17)/8/(1-.981)

11.105263157894736